In [530]:
# %pip install pandas
# %pip install catboost
# %pip install scikit-learn

In [562]:
import pandas as pd
from catboost import CatBoostRanker, Pool
from sklearn.model_selection import StratifiedGroupKFold
from time import time

RANDOM_STATE = 42

# Анализ данных

In [532]:
# Загрузим данные
df_train = pd.read_csv('data/train_df.csv')
df_test = pd.read_csv('data/test_df.csv')

In [533]:
# Удалим дубликаты
df_train.drop_duplicates(inplace=True)

In [534]:
#Посмотрим на датасет
df_train.sample(10)

,search_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,target
494,68329,9,0,0,1,20,4,13,0,2,...,0.256151,0.819359,0.209880,0,0,0,0.70751,0.0,0.0,0
5858,95736,9,0,0,1,20,4,43,0,6,...,0.249243,0.328518,0.081881,0,0,0,0.10866,0.0,0.0,0
12269,223642,9,0,0,1,20,4,25,0,2,...,0.220518,0.407928,0.089955,0,0,0,0.33472,0.0,0.0,0
8078,323285,9,0,0,1,20,4,40,0,3,...,0.197140,0.408737,0.080579,0,0,0,0.88694,0.0,0.0,0
8986,417635,9,0,6,1,20,4,36,0,2,...,0.183096,0.401529,0.073518,0,0,0,0.68908,0.0,0.0,0
8927,410765,9,0,0,0,9,4,47,0,0,...,0.279491,0.522791,0.146115,0,0,0,0.07308,0.0,0.0,1
13038,287085,9,0,0,1,20,4,46,0,2,...,0.193692,0.474438,0.091895,0,0,0,0.07308,0.0,0.0,0
5007,9450,9,1,1,1,20,4,38,2,3,...,0.338540,0.638717,0.216231,0,0,0,0.54849,0.0,0.0,0
10857,93233,9,0,0,1,20,4,25,0,0,...,0.178884,0.543217,0.097173,0,0,0,0.27376,0.0,0.0,0
2517,267916,9,0,0,1,20,4,34,0,0,...,0.271808,0.602927,0.163880,0,0,0,0.07308,0.0,0.0,0


Оценки ранжирования представлены бинарными значениями 0 и 1.

In [535]:
df_train.describe()

,search_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,target
count,15069.000000,15069.0,15069.00000,15069.000000,15069.000000,15069.000000,15069.000000,15069.000000,15069.000000,15069.000000,...,15069.000000,15069.000000,15069.000000,15069.0,15069.0,15069.0,15069.000000,15069.000000,15069.000000,15069.000000
mean,254801.947641,9.0,0.19351,0.463999,0.630168,15.931847,3.876302,33.807685,1.210764,1.792023,...,0.199704,0.426882,0.096538,0.0,0.0,0.0,0.297702,0.030567,0.031232,0.021368
std,142377.307514,0.0,0.62605,1.182782,0.482775,5.310526,0.469923,11.312467,2.234444,1.883771,...,0.088580,0.247193,0.063289,0.0,0.0,0.0,0.311973,0.113026,0.115641,0.144614
min,116.000000,9.0,0.00000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
25%,134823.000000,9.0,0.00000,0.000000,0.000000,9.000000,4.000000,26.000000,0.000000,0.000000,...,0.168716,0.255757,0.052735,0.0,0.0,0.0,0.036740,0.000000,0.000000,0.000000
50%,255859.000000,9.0,0.00000,0.000000,1.000000,20.000000,4.000000,34.000000,0.000000,2.000000,...,0.212826,0.431185,0.091782,0.0,0.0,0.0,0.176140,0.000000,0.000000,0.000000
75%,376175.000000,9.0,0.00000,0.000000,1.000000,20.000000,4.000000,44.000000,1.000000,3.000000,...,0.253881,0.603934,0.136083,0.0,0.0,0.0,0.529630,0.000000,0.000000,0.000000
max,496975.000000,9.0,3.00000,6.000000,1.000000,20.000000,6.000000,49.000000,7.000000,7.000000,...,0.485510,0.999072,0.411563,0.0,0.0,0.0,1.000020,0.846272,0.908132,1.000000


In [630]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15069 entries, 0 to 15080
Data columns (total 77 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   search_id   15069 non-null  int64  
 1   feature_1   15069 non-null  int64  
 2   feature_2   15069 non-null  int64  
 3   feature_3   15069 non-null  int64  
 4   feature_4   15069 non-null  int64  
 5   feature_5   15069 non-null  int64  
 6   feature_6   15069 non-null  int64  
 7   feature_7   15069 non-null  int64  
 8   feature_8   15069 non-null  int64  
 9   feature_9   15069 non-null  int64  
 10  feature_10  15069 non-null  int64  
 11  feature_11  15069 non-null  int64  
 12  feature_12  15069 non-null  int64  
 13  feature_13  15069 non-null  int64  
 14  feature_14  15069 non-null  int64  
 15  feature_15  15069 non-null  int64  
 16  feature_16  15069 non-null  float64
 17  feature_17  15069 non-null  float64
 18  feature_18  15069 non-null  float64
 19  feature_19  15069 non-null  fl

В train датасете присутствуют строки с константным значением.

In [536]:
df_train.nunique()[df_train.nunique() == 1]

feature_0     1
feature_73    1
feature_74    1
feature_75    1
dtype: int64

In [537]:
# Удалим их
df_train.drop(columns=['feature_0', 'feature_73', 'feature_74', 'feature_75'],
              inplace=True)

In [538]:
# Посмотрим, сколько на каждый запрос может быть рекомендовано документов
df_train.groupby('search_id').sum()['target'].unique().tolist()

[0, 1, 2, 4, 3, 5]

В обучающей выборке наблюдается от 0 до 5 рекомендованных значений.

In [539]:
# Посмотрим на баланс классов
100 * len(df_train[df_train['target'] == 0]['target'])/len(df_train)

97.86316278452452

Наблюдается сильный дисбаланс рекоммендаций - почти 98% значений не рекомендовано к выдаче.

In [540]:
# Посмотрим на признаки, которые имеют до 10 уникальных значений
probably_categorical_list = df_train.nunique()[df_train.nunique() < 10].index.tolist()[:-1:]
df_train[probably_categorical_list].sample(10)

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
6742,0,0,1,20,4,0,0,1,0,0,0,0,0,0
12851,0,1,0,9,4,0,2,1,0,0,0,0,0,0
13344,0,0,1,20,4,0,2,1,0,0,0,0,1,0
5840,0,0,1,20,4,0,0,1,0,0,0,0,0,0
10765,0,0,0,9,4,0,4,1,0,0,0,0,1,0
14821,0,0,0,9,4,0,4,1,0,1,1,0,0,0
4958,0,5,0,9,4,0,0,1,0,0,0,0,1,0
2583,0,0,0,9,4,0,3,1,1,0,1,0,1,0
1141,0,0,1,20,4,0,1,0,0,1,0,0,0,0
5728,0,0,0,9,4,0,3,1,0,0,0,1,1,0


Признаки, которые имеют до 10 уникальных значений и тип int64, скорее всего являются категориальными
Датасет выглядит чисто, очевидных ошибок не наблюдается.

In [541]:
# Аналогично обработаем test
df_test.drop(columns=['feature_0', 'feature_73', 'feature_74', 'feature_75'],
             inplace=True)

# Построение модели

Для создания модели ранжирования воспользуемся CatBoostRanker.

In [631]:
# Отделим таргеты
X = df_train.copy()
y = X.pop('target')

X_test = df_test.copy()
y_test = X_test.pop('target')

Создадим валидационный датасет для подбора гиперпараметров. Используем стратификацию и GroupKFold для того, чтобы обеспечить одинаковое распределения таргета в тестовой и валидационной выборках, а также для того, чтобы запросы (queries) не разбивались во время сплита и целиком входили либо в train, либо в val.

In [547]:
# Создадим валидационный датасет для подбора гиперпараметров
sgkf = StratifiedGroupKFold(n_splits=5)

groups = X['search_id'].tolist()
g = sgkf.split(X, y, groups)

# Берем первый фолд - так обеспечивается распределение 80/20
# val включает в себя ~3к значений, что вдвое больше теста
train_idx, val_idx = next(g)
X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

In [632]:
# Удалим search_id
queries_train = X_train['search_id'].tolist()
queries_val = X_val['search_id'].tolist()
queries_test = X_test['search_id'].tolist()

X_train.pop('search_id')
X_val.pop('search_id')
X_test.pop('search_id')
pass

KeyError: 'search_id'

In [549]:
# Создадим пулы - структуру данных, которая используется в CatBoost
train_pool = Pool(data=X_train,
                  label=y_train,
                  group_id=queries_train,
                  cat_features=probably_categorical_list)
val_pool = Pool(data=X_val,
                label=y_val,
                group_id=queries_val,
                cat_features=probably_categorical_list)
test_pool = Pool(data=X_test,
                 label=y_test,
                 group_id=queries_test,
                 cat_features=probably_categorical_list)

Посмотрим, какая из функций потерь показывает лучший результат метрики NDCG.

In [566]:
loss_functions = ['StochasticFilter', 'PairLogit', 'PairLogitPairwise', 'LambdaMart',
                  'QueryRMSE', 'QuerySoftMax', 'YetiRank', 'YetiRankPairwise',]
results = []

# Обучим несколько ранжирующих моделей
for loss_function in loss_functions:
    t0 = time()
    rnkr = CatBoostRanker(custom_metric=['NDCG'],
                          random_state=RANDOM_STATE,
                          iterations=500,
                          loss_function=loss_function,
                          verbose=False)

    rnkr.fit(train_pool, eval_set=val_pool)
    score = round(rnkr.score(val_pool), 3)
    print(f'{loss_function}, NDCG={score}, t={round(time()-t0, 2)}')
    results.append((loss_function, score))

StochasticFilter, NDCG=0.88, t=8.34
PairLogit, NDCG=0.881, t=5.38
PairLogitPairwise, NDCG=0.869, t=59.29
LambdaMart, NDCG=0.888, t=1.99
QueryRMSE, NDCG=0.875, t=1.94
QuerySoftMax, NDCG=0.88, t=13.07
YetiRank, NDCG=0.884, t=2.79
YetiRankPairwise, NDCG=0.881, t=25.62


Наилучший результат показывает модель, использующая LambdaMart.

In [573]:
# Обучим модель с лоссом LambdaMart
rnkr = CatBoostRanker(custom_metric=['NDCG'],
                      random_state=RANDOM_STATE,
                      iterations=500,
                      loss_function='LambdaMart',
                      verbose=50)

rnkr.fit(train_pool, eval_set=val_pool)

Groupwise loss function. OneHotMaxSize set to 10
0:	test: 0.8525803	best: 0.8525803 (0)	total: 3.75ms	remaining: 1.87s
50:	test: 0.8827896	best: 0.8836569 (31)	total: 177ms	remaining: 1.56s
100:	test: 0.8850166	best: 0.8881081 (89)	total: 354ms	remaining: 1.4s
150:	test: 0.8843020	best: 0.8881081 (89)	total: 569ms	remaining: 1.31s
200:	test: 0.8849481	best: 0.8881081 (89)	total: 765ms	remaining: 1.14s
250:	test: 0.8834978	best: 0.8881081 (89)	total: 976ms	remaining: 969ms
300:	test: 0.8822042	best: 0.8881081 (89)	total: 1.19s	remaining: 790ms
350:	test: 0.8851783	best: 0.8881081 (89)	total: 1.38s	remaining: 585ms
400:	test: 0.8834402	best: 0.8881081 (89)	total: 1.56s	remaining: 384ms
450:	test: 0.8836695	best: 0.8881081 (89)	total: 1.73s	remaining: 188ms
499:	test: 0.8838802	best: 0.8881081 (89)	total: 1.91s	remaining: 0us

bestTest = 0.8881081248
bestIteration = 89

Shrink model to first 90 iterations.


# Предсказание на тестовой выборке

Расчитаем NDCG на тесте и сохраним предсказание в файл.

In [578]:
# NDCG на тестовом датасете
round(rnkr.score(test_pool), 2)

0.91

In [629]:
# Добавим ранжирование в тестовый датасет
predictions = pd.concat([df_test['search_id'],
                        pd.DataFrame(rnkr.predict(test_pool),
                                     columns=['ranking']),
                        df_test['target']], axis=1)
predictions

,search_id,ranking,target
0,10655.0,-0.470215,0.0
1,10655.0,-0.456435,0.0
2,10655.0,-0.428377,0.0
3,10655.0,-0.376507,0.0
4,10655.0,-0.448321,0.0
...,...,...,...
1527,493078.0,NaN,1.0
1528,493078.0,NaN,0.0
853,NaN,-0.082269,NaN
857,NaN,-0.325942,NaN


In [ ]:
# Сохраним в файл
predictions.to_csv('predictions.txt', header=True, index=False)